In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../HotspotAnalysis/")
import os
from AnalysisFunctions import *
from ROIFunctions import *
from PlottingFunctions import *
import cv2
import numpy as np
from imageio import imwrite
import holoviews as hv
from holoviews import opts
from skimage.measure import block_reduce
from IPython.display import display
hv.notebook_extension('bokeh')
#from IPython.display import display, HTML
#display(HTML("""
#<style>
#.output {
#    display: flex;
#    align-items: center;
#    text-align: center;
#    }
#</style>
#"""))

## Load sample image

In [ ]:
imgPath = "."
imgName = "S1_PFC_Adult1b_sectionB4R.czi"
savePath = None

img, DAPI = loadImage(imgPath, imgName, img_channel=2, DAPI_channel=0)
name = imgName.split(".")[0]
opts.defaults(
    opts.GridSpace(shared_xaxis=True, shared_yaxis=True),
    opts.Image(cmap='gist_gray', width=int(img.shape[1]/3), height=int(img.shape[0]/3)),
    opts.Labels(text_color='white', text_font_size='8pt', text_align='left', text_baseline='bottom'),
    opts.Path(color='white'),
    opts.Spread(width=600),
    opts.Overlay(show_legend=False))

%output size=100
allChannels = np.dstack((DAPI, img))
ds = hv.Dataset((['DAPI','Image'], np.arange(DAPI.shape[1]), np.arange(DAPI.shape[0]), allChannels), ['Channel','X', 'Y'], 'Intensity')
allChannelsPlot = ds.to(hv.Image, ['X', 'Y'], dynamic=True)
allChannelsPlot.opts(invert_yaxis=True)

## Remove saturated pixels

In [ ]:
#%%output size=100
if type(img[0,0]) == np.uint8:
    threshold = 200
elif type(img[0,0]) == np.uint16:
    threshold = 50000
else:
    raise Exception("img is not 8- or 16-bit")
imgNew = img.copy()
avgPixel = np.mean(img)
imgNew[imgNew > threshold] = avgPixel
thresholded = np.dstack((img, imgNew))
ds = hv.Dataset((np.arange(2), np.arange(img.shape[1]), np.arange(img.shape[0]), thresholded), ['Threshold','X', 'Y'], 'Intensity')
thresholdedPlots = ds.to(hv.Image, ['X', 'Y'], dynamic=True)
thresholdedPlots.opts(invert_yaxis=True)

In [ ]:
img = imgNew

In [ ]:
roiPath = os.path.join(imgPath, '{}_ROI.tif'.format(imgName.split('.')[0]))
mask, croppedImg, maskedImage, ROIimage, processedImage = load_ROI(img, DAPI, roiPath)
#del DAPI

## Create processed image of randomized pixels

In [ ]:
randArray = maskedImage.copy()
random_pixels = np.random.choice(a=maskedImage[~maskedImage.mask].data, size=maskedImage[~maskedImage.mask].data.size, replace=False)
randArray[~randArray.mask] = random_pixels

In [ ]:
# from tqdm import tqdm_notebook
# for i, val in enumerate(tqdm_notebook(random_pixels)):
#     rand_array[~rand_array.mask][i] = val

In [ ]:
maskedImage.shape

In [ ]:
%output size=100
allChannels = np.dstack((maskedImage.filled(-1), randArray.filled(-1)))
ds = hv.Dataset((['Image','Randomized'], np.arange(maskedImage.shape[1]), np.arange(maskedImage.shape[0]), allChannels), ['Channel','X', 'Y'], 'Intensity')
allChannelsPlot = ds.to(hv.Image, ['X', 'Y'], dynamic=True)
allChannelsPlot.opts(opts.Image(tools=['hover'], invert_yaxis=True, width=int(maskedImage.shape[1]), height=int(maskedImage.shape[0])))

## Run hotspot analysis on both actual image and randomized image

In [ ]:
%%time
nx = 20
ny = 20
FULLname = name + "_" + str(nx) + "x" + str(ny)
stats_actual = Getis(mask, maskedImage, nx, ny)
direction_actual, zs_actual, DL_actual, VL_actual, DM_actual, VM_actual, MLaxisZs_actual, DVaxisZs_actual, quadrantStds_actual = processedStats(stats_actual, maskedImage, FULLname)

In [ ]:
%%time
nx = 20
ny = 20
FULLname = name + "_" + str(nx) + "x" + str(ny)
stats_rand = Getis(mask, randArray, nx, ny)
direction_rand, zs_rand, DL_rand, VL_rand, DM_rand, VM_rand, MLaxisZs_rand, DVaxisZs_rand, quadrantStds_rand = processedStats(stats_rand, randArray, FULLname)

In [ ]:
%output size=100
scale_rand_plot = True
allChannels = np.dstack((zs_actual, zs_rand))
ds = hv.Dataset((['Image','Randomized'], np.arange(zs_actual.shape[1]), np.arange(zs_actual.shape[0]), allChannels), ['Channel','X', 'Y'], 'Intensity')
allChannelsPlot = ds.to(hv.Image, ['X', 'Y'], dynamic=True)
if scale_rand_plot:
    display(allChannelsPlot.opts(opts.Image(tools=['hover'],
                                            cmap='Spectral_r', colorbar=True, invert_yaxis=True,
                                            width=int(maskedImage.shape[1]), height=int(maskedImage.shape[0]),
                                            clim=(zs_actual[~np.isnan(zs_actual)].min(),zs_actual[~np.isnan(zs_actual)].max()))))
else:
    display(allChannelsPlot.opts(opts.Image(tools=['hover'],
                                            width=int(maskedImage.shape[1]), height=int(maskedImage.shape[0]),
                                            cmap='Spectral_r', colorbar=True, invert_yaxis=True)))

In [ ]:
Heatmap_actual = HeatmapPlot(zs_actual)
Heatmap_rand = HeatmapPlot(zs_rand)
# Heatmap_actual.savefig(((FULLname + "actual_Heatmap_Plot.pdf")), bbox_inches="tight")
# Heatmap_rand.savefig(((FULLname + "rand_Heatmap_Plot.pdf")), bbox_inches="tight")

In [ ]:
freq_actual, edges_actual = np.histogram(zs_actual[~np.isnan(zs_actual)], 100)
freq_rand, edges_rand    = np.histogram(zs_rand[~np.isnan(zs_rand)], 20)

color = 'steelblue'
xlabel = 'Z Scores'
alpha = 0.8
line_width = 0.1
plot_width = 800
aspect = 2
(
 hv.Histogram((edges_actual, freq_actual)).opts(xlabel=xlabel, width=plot_width, aspect=aspect, color=color, alpha=alpha, line_width=line_width) +
 hv.Histogram((edges_rand, freq_rand)).opts(xlabel=xlabel, width=plot_width, aspect=aspect, color=color, alpha=alpha, line_width=line_width)
)

In [ ]:
DV_ML_actual = DV_ML_Plot(stats_actual, zs_actual, MLaxisZs_actual, DVaxisZs_actual, direction_actual)
DV_ML_rand = DV_ML_Plot(stats_rand, zs_rand, MLaxisZs_rand, DVaxisZs_rand, direction_rand)
# DV_ML_actual.savefig(((FULLname + "actual_DV_ML_Plot.pdf")), bbox_inches="tight")
# DV_ML_rand.savefig(((FULLname + "rand_DV_ML_Plot.pdf")), bbox_inches="tight")

In [ ]:
Quadrant_actual = QuadrantPlot(stats_actual, maskedImage, DL_actual, VL_actual, DM_actual, VM_actual)
Quadrant_rand = QuadrantPlot(stats_rand, randArray, DL_rand, VL_rand, DM_rand, VM_rand)
# Quadrant_actual.savefig(((FULLname + "actual_Quadrant_Plot.pdf")), bbox_inches="tight")
# Quadrant_rand.savefig(((FULLname + "rand_Quadrant_Plot.pdf")), bbox_inches="tight")